In [11]:
import pandas as pd

import pyproj
from shapely.geometry import shape
from shapely.ops import transform
import ast
# Loading JSON file from the website https://tree-nation.com/de/projekte
data_treenation = pd.read_json('Forest_Monitoring/input/TN/treenation.json')


data_treenation.to_csv('Tree_Nation.csv', index=False)

In [12]:
df=pd.read_csv("Tree_Nation.csv",encoding='latin-1')
df.head()

,address,created_at,description,id,image,name,polygon_data,project_id,slug
0,,2007-05-01 12:09:33+00:00,La plantation de Dosso est la plantation princ...,1,https://treenation-uploads.s3.eu-central-1.ama...,Dosso Centre forestier,NaN,1,dosso-centre-forestier
1,,2008-02-01 13:10:20+00:00,PÃ©piniÃ¨re situÃ©e dans une zone aride. Cette...,2,https://treenation-uploads.s3.eu-central-1.ama...,Bonkoukou,NaN,1,bonkoukou
2,,2008-02-01 14:11:52+00:00,"Dans la commune rurale de Kourfeye centre, can...",4,https://treenation-uploads.s3.eu-central-1.ama...,Talakia,"[[14.053844216867532,3.3945930004119873],[14.0...",1,talakia
3,,2007-03-01 14:35:21+00:00,Notre premiÃ¨re pÃ©piniÃ¨re situÃ©e Ã l'INRAN...,5,https://treenation-uploads.s3.eu-central-1.ama...,Niamey,NaN,1,niamey
4,,2016-04-08 12:39:11+00:00,El objetivo es el aumento de la cantidad y cal...,10,https://treenation-uploads.s3.eu-central-1.ama...,JardÃ­n del EdÃ©n,NaN,2,jardin-del-eden


In [13]:

# Dropping rows where 'polygon_data' which is geometry coordinate is null
df = df.dropna(subset=['polygon_data'])
print(df["polygon_data"])

2      [[14.053844216867532,3.3945930004119873],[14.0...
5      [[13.006439397843762,-85.23716926574707],[13.0...
6      [[12.928736207096296,-84.9886679649353],[12.92...
7      [[13.051636578094332,-85.47683000564575],[13.0...
8      [[13.098037664333242,-85.24369239807129],[13.0...
                             ...                        
648    [[-2.645702424584874,40.2669372155843],[-2.645...
649    [[-2.324134003678672,40.76201154179547],[-2.32...
650    [[-2.533989657793934,40.53891065096518],[-2.53...
651    [[-3.323177538490596,39.97148927893921],[-3.32...
652    [[52.38263086733876,-98.91391526452637],[52.37...
Name: polygon_data, Length: 598, dtype: object


In [14]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import ast

def swap_lat_lon(coords):
    """Swap the positions of latitude and longitude in a list of coordinates."""
    return [(lon, lat) for lat, lon in coords]

def convert_to_list(data):
    if isinstance(data, list):
        return swap_lat_lon(data)
    try:
        if data.endswith(","):
            data = data[:-1]
        coords = ast.literal_eval(data)
        return swap_lat_lon(coords)
    except SyntaxError:
        print(f"Failed to convert string to list: {data}")
        return None

# Applying the changes to the DataFrame
df['polygon_data'] = df['polygon_data'].apply(convert_to_list)
df = df[df['polygon_data'].notna()]

# Creating Polygons from the coordinates
df['polygon_data'] = df['polygon_data'].apply(lambda x: Polygon(x) if len(x) >= 4 else None)
df = df[df['polygon_data'].notna()]

# Converting DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='polygon_data')

# Saving as GeoJSON
gdf.to_file("Tree_nation.geojson", driver='GeoJSON')

# Saving as shapefile
gdf.to_file("Tree_nation.shp")


Failed to convert string to list: [[47.84744335686368,-93.99277486064446],[47.8474475967424,-93.99436354110921],[47.84747718632616,-93.99984143271416],[47.8474918961176,-94.00258589351824],[47.84750235596893,-94.00454202409131],[47.84750650590985,-94.005319324319],[47.84753557549342,-94.01079720592365],[47.84755404522684,-94.01430390695083],[47.84755612514056,-94.01543223728061],[47.84755766507605,-94.01627532752711],[47.84759847466999,-94.02163745909975],[47.84763902426383,-94.02699960067223],[47.84767932385752,-94.03236172224459],[47.84771937345119,-94.0377238938168],[47.84762650302427,-94.0431273853785],[47.84753337259731,-94.04853084693997],[47.84747141231384,-94.05211757797638],[47.84744000217024,-94.05393429850133],[47.84740184199613,-94.05613733913785],[47.84734636174312,-94.0593377000625],[47.84727459131972,-94.06472934162387],[47.84720258089615,-94.07012098318505],[47.84713030047259,-94.07551257474609],[47.84705845005299,-94.08085275629212],[47.84705776004897,-94.0809042063070

<ipython-input-14-87d55d063bc3>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['polygon_data'] = df['polygon_data'].apply(lambda x: Polygon(x) if len(x) >= 4 else None)
<ipython-input-14-87d55d063bc3>:37: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("Tree_nation.shp")
